In [ ]:
#Installing the required package
pip install networkit

In [ ]:
#Importing required packages
import networkit as nk
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
arel = pd.read_csv(### Asset relationship file directory here ###)

In [ ]:
na_ids = arel.loc[arel[['source_asset_id','target_asset_id']].isnull().any(axis=1)]
arel = arel.drop(labels = na_ids.index, axis = 0)

In [ ]:
#Forming a graph based on the network
unique= pd.unique(arel[['source_asset_id', 'target_asset_id']].values.ravel())
node_to_id = {node: i for i, node in enumerate(unique)}
id_to_node = {i: node for node, i in node_to_id.items()}

G = nk.Graph(n = len(unique), weighted = False, directed = True)

for _, row in arel.iterrows():
    G.addEdge(node_to_id[row['source_asset_id']], node_to_id[row['target_asset_id']])

In [ ]:
#Obtaining centrality scores
deg = [(G.degreeIn(i)+G.degreeOut(i)) for i in range(G.numberOfNodes())]

deg_out = nk.centrality.DegreeCentrality(G, normalized=False, outDeg=True, ignoreSelfLoops=True)
deg_out.run()
degree_out = deg_out.scores()

deg_in = nk.centrality.DegreeCentrality(G, normalized=False, outDeg=False, ignoreSelfLoops=True)
deg_in.run()
degree_in = deg_in.scores()

ev = nk.centrality.EigenvectorCentrality(G)
ev.run()
eigenvector = ev.scores()

cl = nk.centrality.Closeness(G, True, True)
cl.run()
closeness = cl.scores()

btwn = nk.centrality.Betweenness(G)
btwn.run()
betweenness = btwn.scores()

In [ ]:
#Exporting centrality scores
c = pd.DataFrame({
    'name': [id_to_node[i] for i in range(G.numberOfNodes())],
    'degree': deg,
    'in_degree': degree_in,
    'out_degree': degree_out,
    'betweenness': betweenness,
    'closeness': closeness,
    'eigenvector': eigenvector
})

c.to_csv('centrality_scores.csv', index=False)